In [1]:
import os
from openai import OpenAI

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-db3f839bc51e459dae3aab49d1a779e2",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 通过 messages 数组实现上下文管理
messages = [
    {'role': 'user', 'content': '你好'}
]

completion = client.chat.completions.create(
    model="deepseek-r1",  # 此处以 deepseek-r1 为例，可按需更换模型名称。
    messages=messages
)

print("="*20+"第一轮对话"+"="*20)
# 通过reasoning_content字段打印思考过程
print("="*20+"思考过程"+"="*20)
print(completion.choices[0].message.reasoning_content)
# 通过content字段打印最终答案
print("="*20+"最终答案"+"="*20)
print(completion.choices[0].message.content)

messages.append({'role': 'assistant', 'content': completion.choices[0].message.content})
messages.append({'role': 'user', 'content': '你是谁'})
print("="*20+"第二轮对话"+"="*20)
completion = client.chat.completions.create(
    model="deepseek-r1",  # 此处以 deepseek-r1 为例，可按需更换模型名称。
    messages=messages
)
# 通过reasoning_content字段打印思考过程
print("="*20+"思考过程"+"="*20)
print(completion.choices[0].message.reasoning_content)
# 通过content字段打印最终答案
print("="*20+"最终答案"+"="*20)
print(completion.choices[0].message.content)

====================第一轮对话====================
====================思考过程====================
你好！很高兴见到你，有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，都可以告诉我哦！😊
====================最终答案====================
你好！很高兴见到你，有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，都可以告诉我哦！😊
====================第二轮对话====================
====================思考过程====================
好的，用户现在问“你是谁”，看起来他们想更了解我的身份和功能。之前用户先用中文打招呼，我也用中文回复了，然后他们继续用中文提问，所以整个对话都是中文的。

首先，我需要明确用户的需求。用户可能已经知道我是AI助手，但想了解更详细的信息，比如我的开发团队、功能、数据隐私政策等。也有可能他们是在测试我的回答能力，或者对AI技术本身感兴趣。

接下来，我要考虑之前的对话历史。用户第一次打招呼“你好”，我回应了，然后他们直接问“你是谁”。这说明用户可能在初次接触后，想确认我的身份和可信度。这时候需要提供一个全面但简洁的回答，涵盖我的基本信息和能力，同时让用户感到放心。

然后，我需要确保回答准确无误，符合公司的指导方针。要提到我是由中国的深度求索（DeepSeek）公司开发的智能助手，强调多领域的帮助能力，比如回答问题、提供建议、学习辅助等。同时，必须提及数据隐私和安全性，这是用户可能关心的点。

另外，用户可能希望知道我的局限性和更新时间，这样他们可以合理预期我的回答范围。比如，我的知识截止到2023年12月，无法访问实时信息，这些都需要明确说明，避免误解。

还要保持友好和开放的态度，鼓励用户继续提问，让他们感到被欢迎和支持。使用表情符号可以增加亲切感，但根据用户的风格调整，如果之前用户用了表情，可以适当回应。

最后，检查回答是否符合中文表达习惯，避免技术术语过多，确保清晰易懂。同时，结构要条理分明，分点说明，方便用户快速获取信息。

总结来说，回答需要涵盖身份、功能、隐私、局限性，以及鼓励互动，所有内容都要准确、友好且易于理解。
=

In [16]:
import re

def extract_character_features(text):
    prompt = f"""
请阅读以下中文小说文本，并识别所有明确提到的“角色名”及其“特征描述”，并将结果用 JSON 结构返回。
JSON的键值对为"角色名X": "xxx", "特征": "xxx"
例子：
  {{"角色名1": "瑞奇",
  "特征": "25岁黑色短发的男性",
  
  "角色名2": "",
  "特征2": "",
  
  "角色名3": "",
  "特征3": ""}}
注意：
1. 角色名应当真实在文本中出现。
2. 特征包括年龄、外貌、衣着、行为、情绪等明显描述。
3. 不需要编造额外信息。

小说内容如下：
{text}
"""
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key="sk-db3f839bc51e459dae3aab49d1a779e2",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    
    response = client.chat.completions.create(
        model="deepseek-r1",  # 或 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "你是一个擅长文本分析的中文助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"},
        temperature=0.5
    )

    result = response.choices[0].message.content
    
    # 尝试解析 JSON
    try:
        # 步骤 1：去除 Markdown 包裹符
        cleaned = re.sub(r'^```json|```$', '', result.strip(), flags=re.MULTILINE).strip()
        
        # 步骤 2：解析为 Python 对象
        json_data = json.loads(cleaned)
        with open("角色信息.json", "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        return json_data
    except json.JSONDecodeError:
        print("返回内容不是合法的 JSON，可以考虑用正则或人工检查")
        return result


text = "瑞奇是一个25岁的青年，黑色短发，穿着绿色的夹克。他面无表情地走在乡间小路上。艾米躲在一棵大树后面，眼神中透着不安。"
extract_character_features(text)

NameError: name 're' is not defined

In [38]:
from openai import OpenAI
import re
import os
import json
def extract_character_features(text):
    prompt = f"""
请阅读以下中文小说文本，并识别所有明确提到的“角色名”及其“特征描述”，并将结果用 JSON 结构返回。
JSON的键值对为"角色名": "xxx", "特征": "xxx"
注意：
1. 角色名应当真实在文本中出现。
2. 特征仅包括角色的数量、年龄、外貌、衣着，切记不要包含任何人物关系的描述
3. 如果人物的特征不足甚至没有的话，可以根据文本内容编造。
例子：
  {{"角色名1": "帕奇",
  "特征1": "一个20岁的黑发男子",
  
  "角色名2": "",
  "特征2": "",
  
  "角色名3": "",
  "特征3": ""}}

小说内容如下：
{text}
"""
    client = OpenAI(
        # 如果没有配置环境变量，请用百炼API Key替换：api_key="sk-xxx"
        api_key = "sk-db3f839bc51e459dae3aab49d1a779e2",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    
    response = client.chat.completions.create(
        model="qwen-plus",  # 或 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "你是一个擅长文本分析的中文助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"},
        temperature= 0.5
    )

    result = response.choices[0].message.content
    
    # 尝试解析 JSON
    try:
        # 步骤 1：去除 Markdown 包裹符
        cleaned = re.sub(r'^```json|```$', '', result.strip(), flags=re.MULTILINE).strip()
        # 步骤 2：解析为 Python 对象
        json_data = json.loads(cleaned)
        with open("角色信息.json", "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        return json_data
    except json.JSONDecodeError:
        print("返回内容不是合法的 JSON，可以考虑用正则或人工检查")
        return result


text = '''在遥远的一个国度里，住着一个国王和王后，他们渴望有一个孩子。于是很诚意的向上苍祈祷。
“上帝啊！我们都是好国王好王后，请您赐给我们一个孩子吧！”不久以后，王后果然生下了一个可爱的小公主，这个女孩的皮肤白得像雪一般，双颊红得有如苹果，头发乌黑柔顺，因此，国王和王后就把她取名为“白雪公主”。
全国的人民都为白雪公主深深祝福。
白雪公主在国王白雪公主和王后的宠爱之下，逐渐长大了，终于成了一个人见人爱的美少女。白雪公主非常善良、有爱心、她经常和动物一起玩耍。森林的动物，像小鹿、小兔子、松鼠、小鸟都喜欢白雪公主，因为白雪公主会给它们吃食物，还会讲故事给它们听。个性善良犹如天使般的白雪公主，过着幸福快乐的生活。
可是，好景不长，白雪公主的母亲生病去世了。
国王为了白雪公主就迎娶了一位新王后，可是，这位新王后却是个精通法术的女巫。她虽然很美丽，但是个性很骄傲、暴躁。尤其她最恨别人比她美丽。
“这是你新母后！”
当国王向白雪公主介绍新王后时，她还正为死去的母后感到悲伤呢。新王后有一面很奇特的镜子，从镜子里可以得到一切你想知道的答案。
所以，王后经常对着镜子问：“魔镜、魔镜，谁是世界上最美丽的女人？”
“全世界最美的女人就是你，王后。”
可是，有一天，当王后再问魔镜同样的问题时，魔镜却回答说：
“现在白雪公主比你美丽。”
新王后听了非常生气。
“可恶，怎么白雪公主和七个小矮人可以有人比我更美丽，我一定要把她除去。”
于是，她就命令宫廷的武士说：
“我不想再看到白雪公主了，你找个借口，把她带到森林里偷偷杀掉。杀了以后，把她的心和舌头带回来，做为你杀死她的证据。听到没有？不可以有差错……”
“是的，王后……”
武士听了这话之后，就真的把白雪公主带到森林里去了。
当武士抽出刀来杀公主的时候，他看到正在采花的公主，纯洁，善良，犹如天使一般，武士不忍心杀她，就向白雪公主说：
“皇后命令我杀掉公主你，可是我实在狠不下心，所以你还是往森林里逃走吧！”
说完武士见到一头猪，就跑过去宰了它，并取下心和舌头以作为证据。
之后，武士便回皇宫去了。
听到猫头鹰叫声的白雪公主，越走越觉得森林好可怕。
突然，眼前有一栋小木屋，于是便又惊又喜的叫着：
“啊，是小木屋！”
白雪公主急忙向前敲敲门，可是屋子里没有人来开门。
她就自作主张的把门打开。进入小木屋后，里面竟然整齐排列着七张小小的床。　
白雪公主在森林里跑了一天，觉得非常疲倦，就在那七张小小的床上躺了下来，不知不觉的睡着了。
傍晚，当白雪公主和七个小矮人七个小矮人扛着锄头回来时，发现自己的家有人在，而且是睡在自己的床上，大家都很奇怪的问：
“这个漂亮的女孩子是谁啊？”
“她睡得好香哪！”
“这个小姑娘长得真美丽。”
小矮人们纷纷议论的声音吵醒了白雪公主。
小矮人们很生气的说：
“你为什么闯进我们的房子呢？”
“各位先生，真是对不起，因为我在森林中迷路了，走了一整天的路，实在是又饿又累，看见这栋小屋，我就走进来休息了。”
白雪公主又把事情的经过，一五一十地告诉小矮人。小矮人们听了非常同情白雪公主的遭遇，就把她留下来。
“你就在这里住下来吧！”
白雪公主听到小矮人愿意留下她，很高兴的说：
“真是太感谢了，我愿意在这里为你们做饭、铺床、洗衣服、打扫，我什么都愿意为你们做。”
“欢迎你，从此这里就是你的家了。”
白雪公主每天都把这个小木屋打扫得非常清洁。七个小矮人从森林里回来后，就有可口的晚餐等着他们。就这样日复一日，白雪公主和小矮人过着快乐的生活。
新王后以为白雪公主已经死了，有一天她又问魔镜说：
“魔镜、魔镜，谁是世界上最美丽的人呢？”魔镜回答王后说：
“王后，你很美丽，可是白雪公主比你更美丽，她现在在森林中和七个小矮人过着快乐幸福的生活。”
王后听了这个回答之后，才知道白雪公主并没有死，她感到很愤怒。
“真是可恶极了，一定要让白雪公主从世界上消失”
坏心肠的王后想到了一个办法，她在鲜红的苹果外面，涂上了她调配的毒药，准备去毒死白雪公主。
“嘿！嘿！白雪公主只要吃一口这个有毒的苹果，就一定会死去。到那个时候，我就是世界上最美丽的女人了。” 然后，王后就打扮成老太婆的模样，提着一篮苹果到森林里去了。坏王后提着一篮苹果来到了小矮人的小木屋前。
“可爱的小姑娘，你要不要买一个又红又香的苹果呀！我送一个给你吃吧，相信你一定会喜欢的。”
本来就很喜欢吃苹果的白雪公主，看到又红又大的苹果，便高兴地说：
“哇！这红红的苹果多么的可爱呀！一定很好吃的。”
于是白雪公主就伸手接过那个苹果。
结果，白雪公主才咬了一口，就马上倒在地上，昏死过去了。
坏心的王后看到她倒在地上，大笑着说：“哈！哈！白雪公主从此以后就从这个世界上消失了。”
小矮人傍晚回家的时后，看到白雪公主躺在地上像死了一样，他们马上把她抬到床上，尽力的施救，可是白雪公主仍然没有醒过来。
小矮人们哭哭啼啼的把白雪公主，放在一个装满鲜花的玻璃棺材内，准备举行盛大的葬礼。
这时，邻国的王子正好路过森林，看到了玻璃棺材里美丽可爱的公主，还有在旁哀悼的小矮人和小动物们。
王子知道事情的经过之后，含着泪水悲伤地的注视白雪公主说：
“可怜的公主，如果你能复活的话，该有多好呀！”王子向白雪公主献上了花束，含情脉脉的地凝视着她说：“她的皮肤雪白，脸颊红润，好像睡着一般，根本不像死去的人。”
最后，王子情不自禁地俯身吻了她。
突然，白雪公主从口中吐出了吃进去的苹果。原来是王子对公主的爱，使毒苹果失去了效力，公主也逐渐恢复了体温，睁开明亮的双眼。
白雪公主苏醒了过来，好像是从长睡中醒来一般，她的脸颊和唇依旧是那么的红润。
“哇！你们看到了吧！白雪公主活过来了！白雪公主复活了！”
小矮人们都雀跃不已，兴奋地叫着。王子更是满心欢喜地说：
“真是太好了！白雪公主重生了，上帝真的不会让我失望啊！”
连在旁的动物们也吱吱喳喳谈论不休。
王子向白雪公主说明了他的来历之后，就握着公主的双手，温柔的说：“公主，你愿意和我一起回皇宫，做我的王妃吗？”
白雪公主羞怯地点头答应了。
小矮人和森林白雪公主和七个小矮人的动物们，有的手舞足蹈，有的欢声歌唱，为王子和白雪公主歌颂庆贺。
“祝愿王子和公主，永远幸福快乐！”
王子带着白雪公主，骑着白马向小矮人和森林里的动物们告别，他们一直回头向小矮人们挥手说再见：
“小矮人们，感谢你们对我的照顾，我会永远记得你们的。” “祝你们幸福，再见！”
他们离开森林，回到邻国之后，马上受到全国人民的欢迎。在人民的心目中，他们真是天生一对。
坏心的王后自从毒害了白雪公主之后，就以为白雪公主必死无疑，所以非常高兴。
有一天，她满怀自信地的问魔镜：
“魔镜、魔镜，现在世界上最美丽的人应该是我了吧！”
魔镜回答说：
“除了白雪公主以外，你是最美丽的人，但是，在邻国生活的白雪公主比你漂亮千万倍。”
王后听了觉得很奇怪，她想白雪公主不是被她害死了吗？ 可是又想到白雪公主运气一向很好，
“简直是气死人了，我就不相信不能杀掉白雪公主！”
王后原本就是个女巫，于是她骑着魔扫帚，带着魔剑，飞往邻国，准备去除掉她的眼中钉。　
当她飞到邻国的上空时，突然一道闪电朝坏王后打来。
“劈！叭！”
一阵响声把女巫王后从扫帚上打下来。
女巫王后终于受到上帝的处罚，结束了作恶多端的生命。
此时，王子的国家却举国欢腾，因为美丽的白雪公主答应了王子的求婚，正在举行盛大的婚礼。
小矮人和森林的动物们也被邀请来参加婚礼，在全国人民的祝福声中，王子和白雪公主将永远快乐的生活在一起。'''

print(extract_character_features(text))

{'角色名1': '国王', '特征1': '一个渴望有孩子的中年男子，穿着华丽的王室服饰。', '角色名2': '王后', '特征2': '一位美丽、善良的女子，皮肤白皙，衣着高贵优雅，因病去世。', '角色名3': '白雪公主', '特征3': '一位拥有雪白肌肤、红润双颊和乌黑柔顺头发的女孩，逐渐成长为美丽的少女，穿着朴素但整洁的衣服。', '角色名4': '新王后', '特征4': '一位精通法术的女巫，外表美丽，穿着华丽但略显诡异的服饰，拥有一头黑色长发。', '角色名5': '七个小矮人', '特征5': '七个身材矮小的男人，年龄各异，穿着简单的工作服，留着胡须。', '角色名6': '王子', '特征6': '一位年轻英俊的邻国王子，身着华丽的骑士服装，有着健康的肤色和明亮的眼睛。', '角色名7': '武士', '特征7': '一名魁梧的宫廷武士，身穿盔甲，手持利剑，心地善良。'}


In [39]:
def update_config_with_characters(character_data, config_path="..\config.json"):
    # 加载已有 config.json
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    # 清空原有的角色字段（角色名1~10 和 特征1~10）
    for i in range(1, 11):
        config[f"角色名{i}"] = ""
        config[f"特征{i}"] = ""

    # 写入新的角色信息
    for idx, (role_key, feature_value) in enumerate(character_data.items()):
        if "角色名" in role_key:
            role_index = idx // 2 + 1
            config[f"角色名{role_index}"] = feature_value
        elif "特征" in role_key:
            feature_index = idx // 2 + 1
            config[f"特征{feature_index}"] = feature_value

    # 保存回 config.json
    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2, ensure_ascii=False)
    print("✅ config.json 已成功更新。")
    
character_info = extract_character_features(text)
update_config_with_characters(character_info)

✅ config.json 已成功更新。


In [9]:
import spacy
from spacy.cli import download
import json
from collections import defaultdict

# 自动下载模型
download("zh_core_web_sm")
nlp = spacy.load("zh_core_web_sm")  # 中文模型

def extract_characters_and_traits(text, max_characters=8):
    doc = nlp(text)
    print(doc.ents)
    character_data = defaultdict(list)

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name = ent.text
            sent = ent.sent
            for token in sent:
                if token.text != name and token.pos_ in ["ADJ", "NOUN", "VERB", "NUM"]:
                    character_data[name].append(token.text)

    # 整理为 JSON 格式
    result = {}
    for idx, (name, traits) in enumerate(character_data.items()):
        if idx >= max_characters:
            break
        suffix = str(idx + 1) if idx > 0 else ""
        result[f"角色名{suffix}"] = name
        result[f"特征{suffix}"] = "，".join(traits)

    # 补空项
    for i in range(len(character_data), max_characters):
        suffix = str(i + 1) if i > 0 else ""
        result[f"角色名{suffix}"] = ""
        result[f"特征{suffix}"] = ""

    return result

# 示例
text = "瑞奇是一个25岁的青年，黑色短发，穿着绿色的夹克。他面无表情地走在乡间小路上。艾米躲在一棵大树后面，眼神中透着不安。"
result = extract_characters_and_traits(text)
print(json.dumps(result, ensure_ascii=False, indent=2))


✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
(25岁, 艾米)
{
  "角色名": "艾米",
  "特征": "躲，一，棵，后面，眼神，透，不安",
  "角色名2": "",
  "特征2": "",
  "角色名3": "",
  "特征3": "",
  "角色名4": "",
  "特征4": "",
  "角色名5": "",
  "特征5": "",
  "角色名6": "",
  "特征6": "",
  "角色名7": "",
  "特征7": "",
  "角色名8": "",
  "特征8": ""
}
